In [1]:
import numpy as np # linear algebra
import dask.array as da
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
import sys

import matplotlib.pyplot as plt 
import seaborn as sns

import warnings
warnings.simplefilter("ignore")
df_train = pd.read_parquet("train.parquet")
df_train["S_2"] = pd.to_datetime(df_train["S_2"])
df_train = df_train.drop(['S_2'],axis = 1)

In [ ]:
df = df.astype('float32')

In [2]:
df_train

,customer_ID,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.938469,0,0.008724,1.006838,0.009228,0.124035,0.0,0.004709,NaN,...,-1,-1,-1,0,0,0.0,NaN,0,0.000610,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.936665,0,0.004923,1.000653,0.006151,0.126750,0.0,0.002714,NaN,...,-1,-1,-1,0,0,0.0,NaN,0,0.005492,0
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.954180,3,0.021655,1.009672,0.006815,0.123977,0.0,0.009423,NaN,...,-1,-1,-1,0,0,0.0,NaN,0,0.006986,0
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.960384,0,0.013683,1.002700,0.001373,0.117169,0.0,0.005531,NaN,...,-1,-1,-1,0,0,0.0,NaN,0,0.006527,0
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.947248,0,0.015193,1.000727,0.007605,0.117325,0.0,0.009312,NaN,...,-1,-1,-1,0,0,0.0,NaN,0,0.008126,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5531446,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,0.979333,14,0.020818,0.828199,0.003487,0.090743,0.0,0.025139,NaN,...,-1,-1,-1,0,0,0.0,NaN,0,0.001498,0
5531447,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,0.984907,10,0.007209,0.812610,0.005904,0.079886,0.0,0.023691,NaN,...,-1,-1,-1,0,0,0.0,NaN,0,0.008225,0
5531448,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,0.983019,15,0.013151,0.815422,0.003457,0.100503,0.0,0.012343,NaN,...,-1,-1,-1,0,0,0.0,NaN,0,0.006773,0
5531449,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,0.969861,15,0.009855,1.003541,0.005117,0.101802,0.0,0.008578,NaN,...,-1,-1,-1,0,0,0.0,NaN,0,0.001168,0


In [3]:
df_train = df_train.groupby(["customer_ID"]).tail(1).set_index("customer_ID")
df_train

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
customer_ID,,,,,,,,,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.934745,0,0.009382,1.007647,0.006104,0.135021,0.0,0.007174,NaN,NaN,...,-1,-1,-1,0,0,0.000000,NaN,0,0.002970,0
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,0.880519,6,0.034684,1.004028,0.006911,0.165509,0.0,0.005068,NaN,0.060646,...,-1,-1,-1,0,0,0.000000,NaN,0,0.003169,0
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,0.880875,0,0.004284,0.812649,0.006450,NaN,0.0,0.007196,NaN,NaN,...,-1,-1,-1,0,0,0.000000,NaN,0,0.000834,0
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,0.621776,0,0.012564,1.006183,0.007829,0.287766,0.0,0.009937,NaN,0.046104,...,-1,-1,-1,0,0,0.000000,NaN,0,0.005560,0
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,0.871900,0,0.007679,0.815746,0.001247,NaN,0.0,0.005528,NaN,0.044671,...,-1,-1,-1,0,0,0.000000,NaN,0,0.006944,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffff41c8a52833b56430603969b9ca48d208e7c192c6a4081a6acc28cf4f8af7,0.844229,15,0.028515,1.009866,0.001928,0.128707,0.0,0.005893,NaN,0.113053,...,-1,-1,-1,0,0,0.000000,NaN,0,0.003009,0
ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fdd3e5b57cfcbee30286,0.831279,1,0.292360,0.055656,0.006953,NaN,0.0,0.233078,NaN,0.134540,...,-1,-1,-1,0,0,0.000000,NaN,0,0.009230,0
ffff9984b999fccb2b6127635ed0736dda94e544e67e026eee4d20f680639ff6,0.800522,9,0.020563,1.007023,0.000957,0.066648,0.0,0.006314,NaN,0.049778,...,-1,-1,-1,0,0,0.000000,NaN,0,0.000340,0


In [4]:

df_train_labels = pd.read_csv("train_labels.csv")
df_train_labels["target"] = df_train_labels["target"].astype("int8")
df_train_labels


,customer_ID,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0
...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,0
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,1


In [5]:
df_train = df_train.merge(df_train_labels, on="customer_ID", how='left')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 458913 entries, 0 to 458912
Columns: 190 entries, customer_ID to target
dtypes: float32(93), int16(9), int8(87), object(1)
memory usage: 215.8+ MB


In [6]:
tmp = df_train.isna().sum().mul(100).div(len(df_train)).sort_values(ascending=False)

tmp

D_88      99.819792
D_110     99.198541
B_39      99.105495
D_73      99.076296
B_42      98.661620
            ...    
D_78       0.000000
D_79       0.000000
R_8        0.000000
R_9        0.000000
target     0.000000
Length: 190, dtype: float64

In [7]:
missingDF = pd.DataFrame(tmp).reset_index()
drop_cols = missingDF[missingDF[0]>75]["index"].values
print(drop_cols)

['D_88' 'D_110' 'B_39' 'D_73' 'B_42' 'D_134' 'B_29' 'D_76' 'D_132' 'D_42'
 'D_142']


In [8]:
df_train

,customer_ID,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.934745,0,0.009382,1.007647,0.006104,0.135021,0.0,0.007174,NaN,...,-1,-1,0,0,0.000000,NaN,0,0.002970,0,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.880519,6,0.034684,1.004028,0.006911,0.165509,0.0,0.005068,NaN,...,-1,-1,0,0,0.000000,NaN,0,0.003169,0,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.880875,0,0.004284,0.812649,0.006450,NaN,0.0,0.007196,NaN,...,-1,-1,0,0,0.000000,NaN,0,0.000834,0,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.621776,0,0.012564,1.006183,0.007829,0.287766,0.0,0.009937,NaN,...,-1,-1,0,0,0.000000,NaN,0,0.005560,0,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.871900,0,0.007679,0.815746,0.001247,NaN,0.0,0.005528,NaN,...,-1,-1,0,0,0.000000,NaN,0,0.006944,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,0.844229,15,0.028515,1.009866,0.001928,0.128707,0.0,0.005893,NaN,...,-1,-1,0,0,0.000000,NaN,0,0.003009,0,0
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0.831279,1,0.292360,0.055656,0.006953,NaN,0.0,0.233078,NaN,...,-1,-1,0,0,0.000000,NaN,0,0.009230,0,0
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0.800522,9,0.020563,1.007023,0.000957,0.066648,0.0,0.006314,NaN,...,-1,-1,0,0,0.000000,NaN,0,0.000340,0,0
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,0.754129,0,0.015838,0.714486,0.000993,0.408849,0.0,0.050048,NaN,...,-1,-1,1,0,0.949723,0.446255,1,0.002502,2,1


In [9]:
df_train = df_train[['S_3', 'S_5', 'S_6', 'S_7', 'S_8', 'S_11', 'S_12', 'S_13', 'S_15',
       'S_16', 'S_22', 'S_23', 'S_24', 'S_25', 'S_27','R_1', 'R_2', 'R_3', 'R_4', 'R_5', 'R_6', 'R_7', 'R_8', 'R_9', 'R_10',
       'R_12', 'R_13', 'R_24', 'R_26', 'R_27','B_1', 'B_2', 'B_3', 'B_4', 'B_6', 'B_7', 'B_9', 'B_10', 'B_11', 'B_14',
       'B_16', 'B_17', 'B_18', 'B_19', 'B_20', 'B_22', 'B_23', 'B_25', 'B_33',
       'B_37', 'B_38', 'B_40','D_39', 'D_41', 'D_44', 'D_47', 'D_48', 'D_51', 'D_54', 'D_55', 'D_58',
       'D_61', 'D_62', 'D_70', 'D_74', 'D_75', 'D_78', 'D_112','P_2','P_3','P_4','target']]

In [10]:
numeric_data = df_train.select_dtypes(include=[np.number])
categorical_data = df_train.select_dtypes(exclude=[np.number])

numeric_data.shape[1]
categorical_data.shape[1]

0

In [11]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean.fit(numeric_data)
numeric_columns =df_train.select_dtypes(np.number).columns
df_train[numeric_columns] = df_train[numeric_columns].fillna(df_train[numeric_columns].mean())
df_train[numeric_columns] = df_train[numeric_columns].round(3)
df_train.isna().sum()

S_3       0
S_5       0
S_6       0
S_7       0
S_8       0
         ..
D_112     0
P_2       0
P_3       0
P_4       0
target    0
Length: 72, dtype: int64

In [12]:
#categorical_data
#cols = ["D_68", "B_30", "B_38", "D_114", "D_116", "D_117", "D_120", "D_126"]
#df_train[cols] = df_train[cols].apply(pd.to_numeric, errors='coerce')
#df_train.isna().sum()

In [13]:
data = df_train.sample(frac=0.1, random_state=786)
data_unseen = df_train.drop(data.index).reset_index(drop=True)
data.reset_index(drop=True, inplace=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions ' + str(data_unseen.shape))
from pycaret.regression import *
df_train["target"].unique
#data.columns()
data.columns

Data for Modeling: (45891, 72)
Unseen Data For Predictions (413022, 72)


Index(['S_3', 'S_5', 'S_6', 'S_7', 'S_8', 'S_11', 'S_12', 'S_13', 'S_15',
       'S_16', 'S_22', 'S_23', 'S_24', 'S_25', 'S_27', 'R_1', 'R_2', 'R_3',
       'R_4', 'R_5', 'R_6', 'R_7', 'R_8', 'R_9', 'R_10', 'R_12', 'R_13',
       'R_24', 'R_26', 'R_27', 'B_1', 'B_2', 'B_3', 'B_4', 'B_6', 'B_7', 'B_9',
       'B_10', 'B_11', 'B_14', 'B_16', 'B_17', 'B_18', 'B_19', 'B_20', 'B_22',
       'B_23', 'B_25', 'B_33', 'B_37', 'B_38', 'B_40', 'D_39', 'D_41', 'D_44',
       'D_47', 'D_48', 'D_51', 'D_54', 'D_55', 'D_58', 'D_61', 'D_62', 'D_70',
       'D_74', 'D_75', 'D_78', 'D_112', 'P_2', 'P_3', 'P_4', 'target'],
      dtype='object')

In [14]:
df_train

,S_3,S_5,S_6,S_7,S_8,S_11,S_12,S_13,S_15,S_16,S_22,S_23,S_24,S_25,S_27,R_1,R_2,R_3,R_4,R_5,R_6,R_7,R_8,R_9,R_10,R_12,R_13,R_24,R_26,R_27,B_1,B_2,B_3,B_4,B_6,B_7,B_9,B_10,B_11,B_14,B_16,B_17,B_18,B_19,B_20,B_22,B_23,B_25,B_33,B_37,B_38,B_40,D_39,D_41,D_44,D_47,D_48,D_51,D_54,D_55,D_58,D_61,D_62,D_70,D_74,D_75,D_78,D_112,P_2,P_3,P_4,target
0,0.135,0.035,0,0.106,1544,15,0.184,702,6,0.000,0.918,0.132,0.936,0.972,0.929,0.006,0,0,0,0,0.004,0.0,0,-1,0,1.0,0,0,-1,1.006,0.009,1.008,0.007,5,0.150,0.058,0.010,0.326,0.010,0.010,0,0.725,1.000,0,0,0,0.040,0.006,1,0.009,2,0.100,0,0.0,0,0.540,0.192,2,1.0,0.187,0.007,0.228,0.447,0,0,0,0,1.0,0.935,0.629,0.0,0
1,0.166,0.044,0,0.209,1284,14,0.192,290,6,0.007,0.921,0.133,0.931,0.978,0.292,0.007,0,1,0,0,0.008,0.0,0,-1,0,1.0,0,0,-1,1.008,0.035,1.004,0.005,1,0.168,0.028,0.013,0.297,0.015,0.019,0,0.725,1.000,0,0,0,0.015,0.021,1,0.033,2,0.020,6,0.0,0,0.402,0.015,1,1.0,0.036,0.010,0.049,0.234,0,0,0,0,1.0,0.881,0.571,0.0,0
2,0.229,0.002,1,0.222,0,12,0.191,0,8,0.009,0.303,0.133,0.086,0.972,0.309,0.006,0,0,0,0,0.009,0.0,0,-1,0,1.0,0,0,-1,1.003,0.004,0.813,0.007,2,0.184,0.027,0.009,0.296,0.005,0.007,0,0.725,1.000,0,0,0,0.020,0.007,1,0.005,1,0.025,0,0.0,0,0.339,0.080,1,1.0,0.099,0.003,0.138,0.439,0,0,0,0,1.0,0.881,0.629,0.0,0
3,0.288,0.023,0,0.279,528,14,0.055,435,8,0.007,0.932,0.133,0.957,0.974,0.681,0.008,0,2,0,0,0.001,0.0,0,-1,0,1.0,0,0,-1,1.004,0.013,1.006,0.010,0,0.174,0.012,0.021,0.412,0.005,0.017,1,1.007,1.000,0,0,0,0.005,0.014,1,0.007,2,0.031,0,0.0,0,0.414,0.409,1,1.0,0.021,0.009,0.027,0.435,0,0,0,0,1.0,0.622,0.672,0.0,0
4,0.229,0.009,1,0.222,0,13,0.186,0,8,0.008,0.298,0.132,0.082,0.972,0.309,0.001,0,0,0,0,0.004,0.0,0,-1,0,1.0,0,0,-1,1.007,0.008,0.816,0.006,21,0.049,0.160,0.004,0.125,0.001,0.009,0,0.725,0.531,0,0,0,0.145,0.010,1,0.008,1,0.117,0,0.0,0,0.480,0.325,0,1.0,0.254,0.383,0.601,0.423,0,2,2,0,1.0,0.872,0.570,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,0.129,0.262,0,0.088,1544,12,0.281,435,6,0.004,0.949,0.137,0.964,0.974,0.545,0.002,0,7,0,0,0.004,0.0,0,-1,0,1.0,0,0,-1,1.003,0.029,1.010,0.006,4,0.317,0.030,0.170,0.436,0.024,0.155,0,0.725,1.000,0,0,0,0.025,0.271,1,0.028,2,0.008,15,0.0,0,0.215,0.059,2,1.0,0.035,0.009,0.074,0.457,0,0,0,0,1.0,0.844,0.531,0.0,0
458909,0.229,0.010,1,0.222,0,12,0.188,0,8,0.006,0.301,0.139,0.079,0.976,0.309,0.007,0,1,0,0,0.009,0.0,0,-1,0,1.0,0,0,-1,1.007,0.292,0.056,0.233,19,0.012,0.272,0.143,0.021,0.234,0.070,12,0.861,0.098,75,17,0,0.263,0.114,0,0.291,7,0.204,1,0.0,1,0.775,0.171,1,1.0,0.146,0.484,0.730,0.094,1,4,5,0,1.0,0.831,0.562,0.0,0
458910,0.067,0.026,0,0.041,2694,19,0.398,544,4,0.004,0.986,0.141,0.967,0.973,0.424,0.001,0,0,0,0,0.003,0.0,0,-1,0,1.0,0,0,-1,1.009,0.021,1.007,0.006,6,0.190,0.049,0.046,0.303,0.011,0.018,2,0.228,1.000,0,0,0,0.037,0.032,1,0.020,3,0.051,9,0.0,0,0.326,0.114,1,1.0,0.107,0.008,0.136,0.223,0,0,0,0,1.0,0.801,0.616,0.0,0
458911,0.409,0.010,0,0.366,0,14,0.192,0,8,0.004,0.902,0.132,0.906,0.979,0.007,0.001,0,0,0,0,0.003,0.0,0,-1,0,1.0,0,0,-1,1.004,0.016,0.714,0.050,18,0.018,0.303,0.016,0.035,0.003,0.012,6,0.737,0.465,0,3,0,0.264,0.009,0,0.021,3,0.329,0,0.0,1,0.592,0.349,0,1.0,0.297,0.259,0.806,0.313,0,1,2,0,1.0,0.754,0.486,0.0,1


In [15]:
#from sklearn.model_selection import StratifiedKFold
y = df_train['target']
#col = df_train["target"]
X = df_train.loc[:, df_train.columns != 'target']

# Ensemble Learning

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
estimator_list = [ 
          ('RF', RandomForestClassifier()),
          ('KNN', KNeighborsClassifier()),
 #         ('SVM', SVC()), 
          ('GNB', GaussianNB()),
          ('XGB', XGBClassifier())
                 ]

stack_model = StackingClassifier(estimators=estimator_list,final_estimator=LogisticRegression())
stack_model.fit(X_train,y_train)


In [ ]:
y_train_pred = stack_model.predict(X_train)
y_test_pred = stack_model.predict(X_test)



In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

stack_model_train_accuracy = accuracy_score(y_train,y_train_pred)
stack_model_train_f1 = f1_score(y_train,y_train_pred,average = 'weighted')

In [ ]:
print(stack_model_train_accuracy)
print(stack_model_train_f1)


In [ ]:

X_train

In [ ]:
df["S_2"] = pd.to_datetime(df_train["S_2"])
df= df_train.drop(['S_2'],axis = 1)